# Generate the RepCode circuit

In [1]:
from result_saver import SaverProvider

provider = SaverProvider()

In [37]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import get_repcode_layout

DISTANCE = 10
ROUNDS = 10
DEVICE = "ibmq_mumbai"
LOGICAL = '1'

code = RepetitionCodeCircuit(DISTANCE, ROUNDS)
backend = provider.get_backend(DEVICE)
layout = get_repcode_layout(DISTANCE, backend, _is_hex=False)

qc = code.circuit[LOGICAL]                                   

Finding the longest path starting from 27 qubits: 100%|██████████| 27/27 [00:00<00:00, 28757.29it/s]


# Noise model

In [38]:
from qiskit_qec.noise import PauliNoiseModel

def get_noise_model(p1Q, p2Q, pXY, pZ, pRO, pRE):

    error_dict = {
        'reset': {
            "chan": {
                        'i':1-pRE,
                        'x':pRE
                    }
                },
        'measure': {
            "chan": {
                        'i':1-pRO,
                        'x':pRO
                    }
                },
        'h': {
            "chan": {
                        'i':1-p1Q
                    }|
                    {
                        i:p1Q/3
                        for i in 'xyz'
                    }
                },
        #idling error attached to a custom gate "idle_1" (assumed to be the identity acting on a single qubit)
        'idle_1': {
            "chan": {
                        'i':1-pXY,
                        'x':pXY/2,
                        'y':pXY/2
                    }
                },
        #another type of idling error attached to a custom gate "idle_2" (assumed to be the identity acting on a single qubit)
        'idle_2': {
            "chan": {
                        'i':1-pZ,
                        'z':pZ
                    }
                },

        'cx': {
            "chan": {
                        'ii':1-p2Q
                    }|
                    {
                        i+j:p2Q/15
                        for i in 'ixyz'
                        for j in 'ixyz'
                        if i+j!='ii'
                    }
                },
        'swap': {
            "chan": {
                        'ii':1-p2Q
                    }|
                    {
                        i+j:p2Q/15
                        for i in 'ixyz'
                        for j in 'ixyz'
                        if i+j!='ii'
                    }
                }
                }

    noise_model = PauliNoiseModel(fromdict=error_dict)

    return noise_model



# Get counts via stim

In [39]:
from qiskit_qec.utils import get_counts_via_stim

SHOTS = 1e4
noise_model = get_noise_model(p1Q=1e-4, p2Q=6e-3, pXY=1e-4, pZ=1e-4, pRO=1e-1, pRE=0)
counts = get_counts_via_stim(qc, shots=int(SHOTS), noise_model=noise_model)
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
(sorted_counts)

[('1111111010 100010101 000010001 000010001 000011001 000011001 000011000 000011000 000010000 010000000 000000000',
  1),
 ('1111111111 010011011 010011011 000011011 000011011 000101011 000101001 000100001 000100000 000100000 000100000',
  1),
 ('1111111100 011010110 011010110 011010110 011010000 011010000 011010000 011010000 011010000 010010000 010010000',
  1),
 ('1111000111 111111100 011001000 011001100 011000000 011001100 011000000 011001100 000010000 000001100 000001000',
  1),
 ('1111110111 100010110 100000000 100000000 100000000 100000010 100000010 100000010 100000010 100000000 000000000',
  1),
 ('1111111111 001001110 000001110 000001100 000001100 000001100 001001100 001001100 001000100 001000100 000001101',
  1),
 ('1111111111 000101110 100101010 100101010 100101010 100001000 100001000 101001000 101101000 100011000 000000000',
  1),
 ('1011001111 001000100 001010101 001000101 001101101 001110100 000101100 000000100 000011010 100000010 000011010',
  1),
 ('1111111111 000011101 

# counts to IQ

In [40]:
from soft_info import get_repcode_IQ_map, get_KDEs

IQ_map = get_repcode_IQ_map(layout, ROUNDS)
print(IQ_map)

mumbai_calib_job = "cne879d72scg008df4j0" # 21.11.23
kde_dict, scaler_dict = get_KDEs(provider, mumbai_calib_job)

{0: 1, 1: 3, 2: 8, 3: 14, 4: 19, 5: 25, 6: 23, 7: 18, 8: 12, 9: 1, 10: 3, 11: 8, 12: 14, 13: 19, 14: 25, 15: 23, 16: 18, 17: 12, 18: 1, 19: 3, 20: 8, 21: 14, 22: 19, 23: 25, 24: 23, 25: 18, 26: 12, 27: 1, 28: 3, 29: 8, 30: 14, 31: 19, 32: 25, 33: 23, 34: 18, 35: 12, 36: 1, 37: 3, 38: 8, 39: 14, 40: 19, 41: 25, 42: 23, 43: 18, 44: 12, 45: 1, 46: 3, 47: 8, 48: 14, 49: 19, 50: 25, 51: 23, 52: 18, 53: 12, 54: 1, 55: 3, 56: 8, 57: 14, 58: 19, 59: 25, 60: 23, 61: 18, 62: 12, 63: 1, 64: 3, 65: 8, 66: 14, 67: 19, 68: 25, 69: 23, 70: 18, 71: 12, 72: 1, 73: 3, 74: 8, 75: 14, 76: 19, 77: 25, 78: 23, 79: 18, 80: 12, 81: 1, 82: 3, 83: 8, 84: 14, 85: 19, 86: 25, 87: 23, 88: 18, 89: 12, 90: 0, 91: 2, 92: 5, 93: 11, 94: 16, 95: 22, 96: 24, 97: 21, 98: 15, 99: 10}
Specified job execution date: 2023-11-21 19:54:05.610765+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2023-11-21 18:54:05.610765+00:00.


In [41]:
# sample from the kde

kde0, kde1 = kde_dict[10]
scaler = scaler_dict[10]
sample = kde0.sample()


# invert the scaling
sample = scaler.inverse_transform(sample)
print(sample)


IQ_point = complex(sample[0][0], sample[0][1])


print(IQ_point)


[[ 1.57733320e+06 -1.03397046e+08]]
(1577333.2001794698-103397046.41346659j)


In [42]:
import numpy as np
from tqdm import tqdm

# Determine the total number of shots and number of qubits
total_shots = sum(shots for _, shots in sorted_counts)
len_IQ_array = len(IQ_map)

# Preallocate the NumPy array
IQ_memory = np.zeros((total_shots, len_IQ_array), dtype=np.complex128)

# Fill the array
shot_idx = 0
for count_str, shots in tqdm(sorted_counts):
    for _ in range(shots):
        num_spaces = 0
        for IQ_idx, bit in enumerate(count_str[::-1]):  # invert the order of count str to get IQ order
            if bit == ' ':
                num_spaces += 1
                continue
            qubit_idx = IQ_map[IQ_idx - num_spaces]
            [kde0, kde1], scaler = kde_dict[qubit_idx], scaler_dict[qubit_idx]
            if bit == '0':
                sample = scaler.inverse_transform(kde0.sample(1))
                IQ_memory[shot_idx, qubit_idx] = complex(sample[0][0], sample[0][1])
            elif bit == '1':
                sample = scaler.inverse_transform(kde1.sample(1))
                IQ_memory[shot_idx, qubit_idx] = complex(sample[0][0], sample[0][1])
        shot_idx += 1

 33%|███▎      | 3299/10000 [00:11<00:22, 303.61it/s]

In [ ]:
import numpy as np
print(np.array(IQ_memory).shape)

(10000, 100)


# Decode

In [ ]:
from Scratch import create_or_load_kde_grid

grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, mumbai_calib_job, 10, 0.8, other_date=None)

Specified job execution date: 2023-11-21 19:54:05.610765+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2023-11-21 18:54:05.610765+00:00.
Searching for ibmq_mumbai and 23.11.21_10h17_10pts_0.8std


In [ ]:
from datetime import datetime

def get_avgs_from_dict(noise_dict):
    '''
    Takes noise dictionary as an argument then returns length 5
    array defining average noise levels: [init, idle, RO, single-, two-qubit gate]
    '''
    init_avg = np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    idle_avg = np.average([noise_dict[qubit]['idle'] for qubit in noise_dict])
    RO_avg = np.average([noise_dict[qubit]['RO'] for qubit in noise_dict])
    single_avg = 1e-4#np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    two_gate_avg = np.average([noise_dict[qubit]['2-gate'][connection]
                        for qubit in noise_dict
                        for connection in noise_dict[qubit]['2-gate']
                        if connection != 'default'])
    return [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]
# End get_avgs_from_dict


def get_noise_dict_from_backend(backend, layout, date=None):
    '''
    Takes a given backend and a mapping object and returns a noise dictionary
    to pass to a new Cross_Platform_Code/Hex_Code to have accurate backend
    specific noise information based on calibration data.
    '''
    # Pre-reqs
    noise_dict = {}
    round_time = 1000e-9
    if date is None:
        t = None
    else:
        t = datetime(day=int(date[-2:]), month=int(date[-4:-2]), year=int(date[:4]))
    properties = backend.properties(datetime=t)
    # Initializing each qubit with single qubit noise
    for qubit in layout:
        # Defining ROI error
        ROI_error = properties.readout_error(qubit)
        # Defining idle error
        t1 = properties.t1(qubit)
        t2 = properties.t2(qubit)
        t_time = min(t1, t2)
        idle_error = 1 - np.exp(-round_time / t_time)
        # Updating dictionary
        noise_dict[qubit] = {}
        noise_dict[qubit]['init'] = ROI_error
        noise_dict[qubit]['idle'] = idle_error
        noise_dict[qubit]['RO'] = ROI_error
        noise_dict[qubit]['gate'] = 1e-4
        noise_dict[qubit]['2-gate'] = {'default': 1e-2}
    # Two qubit gate noise
    all_qiskit_indexes = layout 
    for pair in backend.coupling_map:
        # Making sure connection is in circuit
        if pair[0] in all_qiskit_indexes and pair[1] in all_qiskit_indexes:
            # Getting two qubit gate error
            two_gate_error = properties.gate_error('cx', pair)
            if two_gate_error > .5:
                two_gate_error = .5
            # # Updating dictionary
            # stim_index_0 = map.get_stim_index(qiskit_index=pair[0])
            # stim_index_1 = map.get_stim_index(qiskit_index=pair[1])
            noise_dict[pair[0]]['2-gate'][pair[1]] = two_gate_error
            noise_dict[pair[1]]['2-gate'][pair[0]] = two_gate_error
    return noise_dict
# End get_noise_dict_from_backend

In [ ]:

date = "2023-10-30 09:45:17.340386+01:00" #md.creation_date[0] # md[0] for log 1
noise_dict = get_noise_dict_from_backend(provider.get_backend(DEVICE), layout, date=None) # takes noise closest to today
print("noise_dict:", noise_dict)
avgs_noise = get_avgs_from_dict(noise_dict)
print("avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]:", avgs_noise)

# takes 4s

noise_dict: {1: {'init': 0.013700000000000045, 'idle': 0.011019354359310785, 'RO': 0.013700000000000045, 'gate': 0.0001, '2-gate': {'default': 0.01, 0: 0.005124406740392301, 2: 0.011228440067447998}}, 3: {'init': 0.023900000000000032, 'idle': 0.01940798451657233, 'RO': 0.023900000000000032, 'gate': 0.0001, '2-gate': {'default': 0.01, 2: 0.00926745508512894, 5: 0.012549958468484279}}, 8: {'init': 0.017100000000000004, 'idle': 0.008809979567899306, 'RO': 0.017100000000000004, 'gate': 0.0001, '2-gate': {'default': 0.01, 5: 0.5, 11: 0.009315612008270802}}, 14: {'init': 0.034599999999999964, 'idle': 0.010063858284353522, 'RO': 0.034599999999999964, 'gate': 0.0001, '2-gate': {'default': 0.01, 11: 0.006609358960321776, 16: 0.010585857469143367}}, 19: {'init': 0.03200000000000003, 'idle': 0.005389477864489067, 'RO': 0.03200000000000003, 'gate': 0.0001, '2-gate': {'default': 0.01, 16: 0.01210962693256587, 22: 0.005658562116803989}}, 25: {'init': 0.011900000000000022, 'idle': 0.00640208368068884

In [ ]:
import pymatching
import stim


circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization= avgs_noise[4], #two-qubit-fidelity,
                                after_reset_flip_probability= 0, #reset error,
                                before_measure_flip_probability= avgs_noise[2], #measurement error,
                                before_round_data_depolarization= 0)#avgs_noise[1]) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [ ]:
import cpp_soft_info

matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

p_data = 6.869e-3 # mean sherbrooke noise
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_memory,
                                           ROUNDS, IQ_map, grid_dict,
                                           processed_scaler_dict, p_data=p_data, p_mixed=p_data/100, #p_mixed=1e-80, for d=30
                                           common_measure=-1, _bimodal=False, merge_strategy = "replace")
print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")

num_errors: 804 out of 10000 shots


In [ ]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_memory,
                                           ROUNDS, IQ_map, grid_dict,
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")


num_errors: 9476 out of 10000 shots


In [ ]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 6.869e-3 # mean sherbrooke ECR error
p_mixed = p_data/1 # Same as weighted
# p_meas = 1e-3
p_meas = 15.900e-2 # random found number


# p_data = -1
# p_mixed = -1
# p_meas = -1

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_memory, 
                                           ROUNDS, IQ_map, grid_dict, processed_scaler_dict,
                                           p_data, p_mixed, p_meas, common_measure=-1)

print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")
         
# takes 1s

num_errors: 9979 out of 10000 shots
